In [2]:
df = spark.sql("SELECT * FROM bing_news_lakehouse.tbl_latestnews")
display(df)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 23d0ed6b-871c-4e47-9485-3fa997044dd4)

In [3]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, , , Waiting, )

In [4]:
#Building the model, by setting column to perform sentiment analysis , kind of analysis, 
# Response column for output of analysis , Error column to handle errors
model =(AnalyzeText().setTextCol("description").setKind("SentimentAnalysis")
                     .setOutputCol("response").setErrorCol("error"))

StatementMeta(, , , Waiting, )

In [5]:
#applying the model to our dataframe and assigning the model to response df
response=model.transform(df)

StatementMeta(, , , Waiting, )

In [6]:
display(response)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, e0f37a5f-45ad-4084-956d-a2bbc1f836d7)

creating sentiment column to extract sentiment from the response column 

In [7]:
from pyspark.sql.functions import col
senti=response.withColumn('sentiment',col("response.documents.sentiment"))

StatementMeta(, , , Waiting, )

In [8]:
display(senti)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 2ad4a4f0-248a-4d5f-8777-4773bbf12138)

Dropping unncecessary columns like error and response

In [9]:
sentiment= senti.drop("response","error")

StatementMeta(, , , Waiting, )

In [ ]:
from pyspark.sql.functions import col, to_date
sentiment=sentiment.withColumn("dateposted",to_date(col("dateposted"),"dd-MMM-yyyy"))

### Finally updating the data into new table and setting INCREMENTAL LOAD

In [11]:
from pyspark.sql.utils import AnalysisException

try:
    name="bing_news_lakehouse.tbl_sentimentnews"
    sentiment.write.format("delta").saveAsTable(name)
except AnalysisException:
    print('Table Already Exists')
    sentiment.createOrReplaceTempView("viewOfsentimentdf")

    spark.sql(f""" MERGE INTO {name} target_table
                USING viewOfsentimentdf source_table

                ON target_table.articleurl=source_table.articleurl

                WHEN MATCHED AND 

                source_table.title <> target_table.title OR
                source_table.category <> target_table.category OR
                source_table.description <> target_table.description OR
                source_table.provider <> target_table.provider OR
                source_table.imageurl <> target_table.imageurl OR
                source_table.dateposted <> target_table.dateposted

                THEN UPDATE SET * 

                WHEN NOT MATCHED THEN INSERT * 
             """)



StatementMeta(, , , Waiting, )

Table Already Exists


In [12]:
display(sentiment)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 09350f11-f6f2-4c7e-b946-ff08d9064fd5)